In [85]:
#ライブラリをインポート
import os #OSに依存する様々な機能を利用するためのモジュール(ファイルやディレクトリ操作など)
import re #正規表現を利用するためのモジュール
import csv  #csvファイルを扱うためのモジュール
import math #数学的計算のためのモジュール
import matplotlib.pyplot as plt #グラフ描画のためのモジュール
import numpy as np  #多次元配列計算のためのモジュール
import pandas as pd #データフレームを扱うためのモジュール
from sklearn.model_selection import train_test_split  #データをトレーニング用とテスト用に分けるためのモジュール
from sklearn.linear_model import LinearRegression #線型回帰
from sklearn.svm import SVC #サポートベクターマシン
from sklearn.ensemble import RandomForestClassifier #ランダムフォレスト
from sklearn.metrics import accuracy_score  #機械学習モデルの性能評価のためのモジュール

In [109]:
#定数を定義
BINS = 4000  #ヒストグラムのビンの数
EPSILON = .00001  #スムージングパラメータ
UPPER_LIMIT = 1.1 #静止区間の上限
LOWER_LIMIT = 0.9 #静止区間の加減
STATIONARY_INTERVALS = 5  #静止区間除去のサンプルの間隔(静止区間が何サンプル連続したら除去するか)
TRAIN_SIZE = 0.8  #ランダムフォレストのトレーニングデータの割合

In [87]:
#ディレクトリ内のデータセットのファイル名と周波数を取得する関数
def get_Hz_and_filename(path: str) -> list[int, str]:
    filename = os.listdir(path) #引数のパスのディレクトリの中のファイル名一覧を取得
    Hz_and_filename=[]  #ファイル名と周波数を格納するリストを宣言

    for file in filename:
        Hz = re.search(r'\d+', file)    #正規表現を用いてファイル名の中で一番最初に出てくる数字(周波数)を取得
        if Hz:  #数字の入っていないファイル名があるとエラーを吐くので、このif文でチェックする
            Hz_and_filename.append([int(Hz.group(0)), file])    #ファイル名と周波数を格納

    return Hz_and_filename

In [88]:
#ファイル名と周波数を分けて出力する関数
def divide_Hz_and_filename(Hz_and_filename: list[int, str]) -> tuple[list[int], list[str]]:
    Hz = []
    filename = []
    for row in Hz_and_filename:
      Hz.append(row[0])
      filename.append(row[1])

    return Hz, filename

In [89]:
#加速度データのCSVファイルから3軸加速度を取得する関数
def get_acceleration(filename: str) -> tuple[list[float], list[float], list[float]]:
    AccX, AccY, AccZ = [], [], []
    with open(filename) as f:
        reader = csv.reader(f)
        for row in reader:
            AccX.append(float(row[2]))
            AccY.append(float(row[3]))
            AccZ.append(float(row[4]))

    return AccX, AccY, AccZ

In [90]:
#静止区間を除去する関数
def remove_stationary_intervals(AccX: list[float], AccY: list[float], AccZ: list[float]) -> list[float]:
    #各軸の加速度の平均を求める
    AvgAccX = sum(AccX) / len(AccX)
    AvgAccY = sum(AccY) / len(AccY)
    AvgAccZ = sum(AccZ) / len(AccZ)

    AvgResultantAcc = math.sqrt(AvgAccX ** 2 + AvgAccY ** 2 + AvgAccZ ** 2) #重力加速度の推定値=合成加速度の平均を求める

    ResultantAcc = [math.sqrt(x ** 2 + y ** 2 + z ** 2) for x, y, z in zip(AccX, AccY, AccZ)]   #各時刻の合成加速度を求める

    #各時刻の合成加速度から静止区間(重力加速度の推定値に近い値が一定以上以上連続している区間)を除去する
    i = 0 #ループ変数
    counter = 0 #静止区間がSTATIONARY_INTERVALS分続いているかをカウントする変数
    while i < len(ResultantAcc):
        if AvgResultantAcc * LOWER_LIMIT < ResultantAcc[i] < AvgResultantAcc * UPPER_LIMIT:   #平均のLOWER_LIMIT倍~UPPER_LIMIT倍の範囲を調べる
            counter += 1    #範囲内ならカウントを増やす
            if counter == STATIONARY_INTERVALS: #カウントがSTATIONARY_INTERVALSに達したらその区間を削除
                del ResultantAcc[i+1-STATIONARY_INTERVALS:i+1]    #スライスでは選択範囲の開始位置startと終了位置stopを[start:stop]のように書くとstart <= x < stopの範囲が選択される #start番目の値は含まれるがstop番目の値は含まれない
                counter = 0 #カウンターをリセット
                i -= STATIONARY_INTERVALS   #削除した分インデックスがズレるので補正する
        else:
            counter = 0 #カウンターをリセット
        i += 1

    return ResultantAcc  #静止区間を除去した後のリストを返す

In [91]:
#連続する2サンプルの差分を取る関数
def differences_of_acceleration(ResultantAcc: list[float]) -> list[float]:
    index = 0
    DifferenceAcc = []
    for dif in ResultantAcc[:-1]:
        DifferenceAcc.append(math.fabs(ResultantAcc[index + 1]*100000 - ResultantAcc[index]*100000))    #100000倍して誤差を取る
        index += 1

    return DifferenceAcc

In [92]:
#KLダイバージェンス関数 #引数として与える2つの分布は非負の値の集合でなければならないことに注意
def KL_divergence(a: list[float], b: list[float]) -> float:
    min_value = min(min(a), min(b)) #a,bの最小値の小さい方
    max_value = max(max(a), max(b)) #a,bの最大値の大きい方

    #a,bのヒストグラムを作成し、同じ数のビンで区切る
    a_hist, _ = np.histogram(a, bins=BINS, range=(min_value, max_value))
    b_hist, _ = np.histogram(b, bins=BINS, range=(min_value, max_value))

    #正規化する(確率分布に変換する、合計を1にする)ために全合計で割る
    a_hist = (a_hist + EPSILON) / a_hist.sum()
    b_hist = (b_hist + EPSILON) / b_hist.sum()

    #KLダイバージェンスの値を返す
    return np.sum([ai * np.log(ai / bi) for ai, bi in zip(a_hist, b_hist)])

In [93]:
#JSダイバージェンス関数 #引数として与える2つの分布は非負の値の集合でなければならないことに注意
def JS_divergence(a: list[float], b: list[float]) -> float:
    min_value = min(min(a), min(b)) #a,bの最小値の小さい方
    max_value = max(max(a), max(b)) #a,bの最大値の大きい方

    #a,bのヒストグラムを作成し、同じ数のビンで区切る
    a_hist, _ = np.histogram(a, bins=BINS, range=(min_value, max_value))
    b_hist, _ = np.histogram(b, bins=BINS, range=(min_value, max_value))

    #正規化する(確率分布に変換する、合計を1にする)ために全合計で割る
    a_hist = (a_hist + EPSILON) / a_hist.sum()
    b_hist = (b_hist + EPSILON) / b_hist.sum()

    #2つの分布の平均値を求める
    mean_hist = (a_hist + b_hist) / 2.0

    #平均とそれぞれの分布のKLダイバージェンスを算出
    kl_a = np.sum([ai * np.log(ai / bi) for ai, bi in zip(a_hist, mean_hist)])
    kl_b = np.sum([ai * np.log(ai / bi) for ai, bi in zip(b_hist, mean_hist)])

    #JSダイバージェンスの値を返す
    return (kl_a + kl_b) / 2.0

In [94]:
#データフレームの各行の中で2番目に小さい値が格納されている場所を調べる関数(最小値は同じ確率分布同士の0.0)
def get_index_and_columns_of_second_smallest(df: pd.DataFrame) -> list[str, str]:
    index_and_columns_of_second_smallest = []  #データフレームの中で2番目に小さい値が格納されている場所のインデックス名とカラム名を格納する変数
    for i in range(len(df)):
        sorted_row = df.iloc[i].sort_values()   #.ilocでデータフレームの要素を行、列の番号の添字で指定する    #各行の要素を昇順に並び替える
        second_smallest_columns = sorted_row.index[1] #各行の2番目に小さい値が格納されているカラム[1]の名前を取得
        #second_smallest_label = df.columns.get_loc(second_smallest_index)
        index_and_columns_of_second_smallest.append((df.index[i], second_smallest_columns))    #インデックスとカラムのラベル名の組を二次元配列に追加
    return index_and_columns_of_second_smallest

In [95]:
#推定精度を算出する巻数
def calculate_accuracy(index_and_columns_of_second_smallest: list[str, str]) -> tuple[float, list[int]]:
    counter = 0
    error_index_list = []
    for i in range(len(index_and_columns_of_second_smallest)):
        #インデックスとカラムのラベル名が同じならばカウンターを1増やす
        if index_and_columns_of_second_smallest[i][0] == index_and_columns_of_second_smallest[i][1]:
            counter += 1
        else:
            error_index_list.append(i)
            print(f"間違ってるやつは{i}番目の{index_and_columns_of_second_smallest[i][0]}と{index_and_columns_of_second_smallest[i][1]}です")

    return (counter / len(index_and_columns_of_second_smallest)) * 100, error_index_list  #精度を100分率で返す

In [96]:
#入力された加速度の差分のリストからヒストグラムを作る関数
def create_histogram(DifferenceAcc_list: list[float]) -> np.histogram:
    min_value = min(map(lambda x:max(x), DifferenceAcc_list))   #入力されたリストの中で最も小さい数
    max_value = max(map(lambda x:max(x), DifferenceAcc_list))   #入力されたリストの中で最も大きい数

    DifferenceAcc_hist = np.zeros((len(DifferenceAcc_list), BINS), dtype=float)

    for i in range(len(DifferenceAcc_list)):
        DifferenceAcc_hist[i], _ = np.histogram(DifferenceAcc_list[i], bins=BINS, range=(min_value, max_value)) #ヒストグラムを作成し、同じ数のビンで区切る
        DifferenceAcc_hist[i] = (DifferenceAcc_hist[i] + EPSILON) / DifferenceAcc_hist[i].sum()     #正規化する(確率分布に変換する、合計を1にする)ために全合計で割る
    return DifferenceAcc_hist

In [97]:
#入力された加速度の差分のリストからヒストグラムを作る関数
def create_histogram2(DifferenceAcc_list: list[float]) -> np.histogram:
    DifferenceAcc_hist = np.zeros((len(DifferenceAcc_list), BINS), dtype=float)

    for i in range(len(DifferenceAcc_list)):
        min_value = min(DifferenceAcc_list[i])
        max_value = max(DifferenceAcc_list[i])
        DifferenceAcc_hist[i], _ = np.histogram(DifferenceAcc_list[i], bins=BINS, range=(min_value, max_value)) #ヒストグラムを作成し、同じ数のビンで区切る
        DifferenceAcc_hist[i] = (DifferenceAcc_hist[i] + EPSILON) / DifferenceAcc_hist[i].sum()     #正規化する(確率分布に変換する、合計を1にする)ために全合計で割る
    return DifferenceAcc_hist

In [98]:
#KLダイバージェンスとJSダイバージェンス算出の一連の流れを自動化した関数
def KL_and_JS(path: str):
    Hz_and_filename = get_Hz_and_filename(path)
    Hz_and_filename.sort(reverse=True)  #周波数の大きい順にソート
    Hz, filename = divide_Hz_and_filename(Hz_and_filename)
    Hz = [str(hz) + "Hz" for hz in Hz]  #周波数の値+"Hz"のリストを作りデータフレームのラベルに用いる

    #使う変数を宣言
    AccX, AccY, AccZ = [], [], []
    ResultantAcc = []
    DifferenceAcc_list = []
    resultKLD = [[0.0 for j in range(len(filename))] for i in range(len(filename))]  # resultKLDの要素を0.0で初期化
    resultJSD = [[0.0 for j in range(len(filename))] for i in range(len(filename))]  # resultKLDの要素を0.0で初期化
    error_index_list = []

    #各データセットからデータを読み込み静止区間を除去したものを二次元配列に格納
    for i in filename:
        AccX, AccY, AccZ = get_acceleration(path+i)
        ResultantAcc = remove_stationary_intervals(AccX, AccY, AccZ)
        DifferenceAcc_list.append(differences_of_acceleration(ResultantAcc))

    #KLダイバージェンスの値を格納
    for i in range(len(filename)):
        for j in range(len(filename)):
            resultKLD[i][j] = KL_divergence(DifferenceAcc_list[i], DifferenceAcc_list[j])

    #JSダイバージェンスの値を格納
    for i in range(len(filename)):
        for j in range(len(filename)):
            resultJSD[i][j] = JS_divergence(DifferenceAcc_list[i], DifferenceAcc_list[j])

    #結果を出力
    df_KLD = pd.DataFrame(resultKLD, index=Hz, columns=Hz)
    display(df_KLD)
    accuracyKLD, error_index_list = calculate_accuracy(get_index_and_columns_of_second_smallest(df_KLD))
    for i in range(len(error_index_list)):
        print(filename[error_index_list[i]])
    print(f"KLダイバージェンスによる推定精度は{accuracyKLD}%です")

    df_JSD = pd.DataFrame(resultJSD, index=Hz, columns=Hz)
    display(df_JSD)
    accuracyJSD, error_index_list = calculate_accuracy(get_index_and_columns_of_second_smallest(df_JSD))
    for i in range(len(error_index_list)):
        print(filename[error_index_list[i]])
    print(f"JSダイバージェンスによる推定精度は{accuracyJSD}%です")

In [99]:
#ランダムフォレストによる機械学習モデル構築と性能評価までを自動化した関数
def random_forest(path: str):
    Hz_and_filename = get_Hz_and_filename(path)
    Hz_and_filename.sort(reverse=True)  #周波数の大きい順にソート
    Hz, filename = divide_Hz_and_filename(Hz_and_filename)

    #使う変数を宣言
    AccX, AccY, AccZ = [], [], []
    ResultantAcc = []
    DifferenceAcc_list = []

    #各データセットからデータを読み込み静止区間を除去したものを二次元配列に格納
    for i in filename:
        AccX, AccY, AccZ = get_acceleration(path+i)
        ResultantAcc = remove_stationary_intervals(AccX, AccY, AccZ)
        DifferenceAcc_list.append(differences_of_acceleration(ResultantAcc))

    DifferenceAcc_hist = create_histogram2(DifferenceAcc_list)
    x_train, x_test, y_train, y_test = train_test_split(DifferenceAcc_hist, Hz, train_size = TRAIN_SIZE, shuffle = True)

    # 学習する
    clf = RandomForestClassifier(random_state=1234)
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    print("正解率 = ", accuracy_score(y_test, y_pred))

In [104]:
path = "all_long_walk_data/"

In [111]:
KL_and_JS(path)

,100Hz,100Hz,100Hz,100Hz,100Hz,100Hz,100Hz,50Hz,50Hz,50Hz,...,50Hz,50Hz,50Hz,10Hz,10Hz,10Hz,10Hz,10Hz,10Hz,10Hz
100Hz,0.000000,0.098418,0.449476,0.394060,0.381124,0.382961,0.113460,0.043436,0.205215,0.176041,...,0.108224,0.041173,0.059306,0.902836,0.209447,0.163438,1.057797,0.240882,1.184499,0.990794
100Hz,0.126542,0.000000,0.197061,0.178786,0.329907,0.311677,0.137479,0.138864,0.067205,0.076407,...,0.334239,0.199771,0.243458,1.136669,0.105450,0.141243,1.141483,0.200180,1.489936,1.234702
100Hz,0.478313,0.189673,0.000000,0.022417,0.392323,0.357885,0.278343,0.507870,0.070317,0.093710,...,0.716909,0.556273,0.599468,1.734679,0.110012,0.293864,1.396168,0.304384,2.160083,1.803532
100Hz,0.422843,0.158694,0.014532,0.000000,0.325748,0.304934,0.227314,0.459633,0.056871,0.072285,...,0.633064,0.493198,0.527076,1.683772,0.087512,0.272912,1.365878,0.282607,2.100192,1.747773
100Hz,0.175409,0.105101,0.294404,0.251287,0.000000,0.018997,0.058104,0.256155,0.190923,0.222652,...,0.282515,0.247665,0.250280,1.427775,0.243813,0.327907,1.534827,0.459815,1.826835,1.574762
100Hz,0.195215,0.128377,0.310038,0.265733,0.025877,0.000000,0.073663,0.276808,0.211393,0.240174,...,0.299360,0.275843,0.275667,1.460503,0.263023,0.351302,1.558971,0.477429,1.864837,1.595534
100Hz,0.074696,0.046169,0.245387,0.201240,0.105168,0.124053,0.000000,0.133881,0.113207,0.117049,...,0.170469,0.133112,0.139677,1.204722,0.145630,0.197163,1.281250,0.288871,1.569749,1.316328
50Hz,0.133776,0.189905,0.551029,0.530565,0.910959,0.762357,0.364663,0.000000,0.255558,0.239275,...,0.376293,0.188248,0.270728,0.734535,0.265308,0.125418,0.865427,0.186897,0.990125,0.806952
50Hz,0.256197,0.081768,0.089232,0.089545,0.447410,0.379497,0.197341,0.252725,0.000000,0.041656,...,0.507956,0.326455,0.378121,1.293633,0.063745,0.135562,1.138200,0.173281,1.665177,1.373717
50Hz,0.230289,0.085725,0.115446,0.105948,0.487143,0.417678,0.206933,0.226846,0.034188,0.000000,...,0.469061,0.285609,0.337610,1.234979,0.053098,0.117199,1.059222,0.131504,1.581920,1.293851


間違ってるやつは0番目の100Hzと50Hzです
間違ってるやつは1番目の100Hzと50Hzです
間違ってるやつは7番目の50Hzと10Hzです
間違ってるやつは12番目の50Hzと100Hzです
間違ってるやつは13番目の50Hzと100Hzです
間違ってるやつは15番目の10Hzと50Hzです
walk100Hz-20230303-111623870.csv
walk100Hz-20230302-165446613.csv
walk50Hz-20230302-165451196.csv
walk50Hz-0803-1229.csv
walk50Hz-0803-1126.csv
walk10Hz-20230302-165445244.csv
KLダイバージェンスによる推定精度は71.42857142857143%です


,100Hz,100Hz,100Hz,100Hz,100Hz,100Hz,100Hz,50Hz,50Hz,50Hz,...,50Hz,50Hz,50Hz,10Hz,10Hz,10Hz,10Hz,10Hz,10Hz,10Hz
100Hz,0.000000,0.021898,0.106764,0.094281,0.049189,0.056771,0.018185,0.011017,0.051104,0.044917,...,0.013828,0.005466,0.005781,0.241154,0.051314,0.043880,0.272265,0.065006,0.284217,0.253218
100Hz,0.021898,0.000000,0.043235,0.036636,0.029013,0.035773,0.010928,0.029954,0.012350,0.014037,...,0.055453,0.033870,0.038285,0.272799,0.018874,0.034420,0.283192,0.050924,0.322395,0.286711
100Hz,0.106764,0.043235,0.000000,0.002163,0.073184,0.075222,0.059713,0.117250,0.016044,0.022218,...,0.153589,0.124107,0.130803,0.359769,0.026136,0.078168,0.339145,0.083743,0.413339,0.375497
100Hz,0.094281,0.036636,0.002163,0.000000,0.063487,0.065766,0.049556,0.106874,0.013662,0.017894,...,0.136621,0.110858,0.116494,0.356315,0.022454,0.074409,0.338765,0.080092,0.410626,0.372396
100Hz,0.049189,0.029013,0.073184,0.063487,0.000000,0.004159,0.015324,0.073778,0.049699,0.058320,...,0.075626,0.069598,0.069406,0.353526,0.064959,0.093009,0.369929,0.124776,0.415469,0.377335
100Hz,0.056771,0.035773,0.075222,0.065766,0.004159,0.000000,0.019468,0.081981,0.054968,0.063567,...,0.082158,0.080039,0.078830,0.363103,0.070384,0.101121,0.379628,0.131809,0.429089,0.388428
100Hz,0.018185,0.010928,0.059713,0.049556,0.015324,0.019468,0.000000,0.037325,0.028579,0.030173,...,0.037594,0.032647,0.033136,0.305915,0.037827,0.057565,0.324113,0.081104,0.363356,0.324912
50Hz,0.011017,0.029954,0.117250,0.106874,0.073778,0.081981,0.037325,0.000000,0.055067,0.047697,...,0.031769,0.011827,0.014518,0.189738,0.049806,0.026488,0.218091,0.043018,0.223577,0.195276
50Hz,0.051104,0.012350,0.016044,0.013662,0.049699,0.054968,0.028579,0.055067,0.000000,0.004820,...,0.094553,0.063775,0.069928,0.284295,0.007905,0.033699,0.279849,0.044321,0.332244,0.297031
50Hz,0.044917,0.014037,0.022218,0.017894,0.058320,0.063567,0.030173,0.047697,0.004820,0.000000,...,0.083918,0.054461,0.060179,0.272378,0.006038,0.029505,0.268138,0.034711,0.316146,0.282370


間違ってるやつは0番目の100Hzと50Hzです
間違ってるやつは7番目の50Hzと100Hzです
間違ってるやつは15番目の10Hzと50Hzです
walk100Hz-20230303-111623870.csv
walk50Hz-20230302-165451196.csv
walk10Hz-20230302-165445244.csv
JSダイバージェンスによる推定精度は85.71428571428571%です


In [115]:
random_forest(path)

正解率 =  0.4


In [114]:
from sklearn.neighbors import KNeighborsClassifier

Hz_and_filename = get_Hz_and_filename(path)
Hz_and_filename.sort(reverse=True)  #周波数の大きい順にソート
Hz, filename = divide_Hz_and_filename(Hz_and_filename)
#使う変数を宣言
AccX, AccY, AccZ = [], [], []
ResultantAcc = []
DifferenceAcc_list = []

#各データセットからデータを読み込み静止区間を除去したものを二次元配列に格納
for i in filename:
    AccX, AccY, AccZ = get_acceleration(path+i)
    ResultantAcc = remove_stationary_intervals(AccX, AccY, AccZ)
    DifferenceAcc_list.append(differences_of_acceleration(ResultantAcc))

DifferenceAcc_hist = create_histogram2(DifferenceAcc_list)
x_train, x_test, y_train, y_test = train_test_split(DifferenceAcc_hist, Hz, train_size = TRAIN_SIZE, shuffle = True)
# 学習する
clf = KNeighborsClassifier()
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print("正解率 = ", accuracy_score(y_test, y_pred))

正解率 =  0.2
